### Fit a regression model

In [1]:
import pandas as pd
df = pd.read_csv('azdiabetes.csv')

In [2]:
import scipy.stats as stats

# Exclude the 'diabetes' variable
X = df.drop(columns=['diabetes', 'glu'])
X['intercept'] = 1
X = X[['intercept', 'npreg', 'bp', 'bmi', 'ped', 'age']].values

y = df['glu']

In [3]:
# Define prior parameters
n = 532
g = n
nu0 = 2
sigma02 = 1

# number of independent samples
s = 1000

In [4]:
import numpy as np
import math

#sample from the distributions
factor = X @ np.linalg.inv(X.T @ X) @ X.T
SSRg = (y.T) @ (np.identity(n) - (g/(g+1)) * factor) @ y
shape = (nu0 + n) / 2
scale = (nu0 * sigma02 + SSRg) / 2
gamma_samples = stats.gamma.rvs(a=shape, scale=scale, size=s)

# Sample variance
sigma = 1 / math.sqrt(sum(gamma_samples)/s)

# Generate independent Monte Carlo samples
mean = (g/(g+1)) * (np.linalg.inv(X.T @ X) @ np.array(X.T) @ y)
cov = (g/(g+1)) * (sum(gamma_samples)/s) * np.linalg.inv(X.T @ X)

mvnormal_samples = np.random.multivariate_normal(mean, cov, s)

beta = sum(mvnormal_samples) / s

### Model Selection by Gibbs sampling

In [ ]:
import numpy as np
from scipy.special import gammaln
import statsmodels.api as sm

# log probability
def lp_y_X(y, X):
    
    n, p = X.shape
    
    if p == 0:
        Hg = 0
        sigma02 = np.mean(y**2)
    else:
        Hg = (g/(g+1)) * factor
        sigma02 = sm.OLS(y, X).fit().mse_resid
    
    SSRg = y.T @ (np.identity(n) - Hg) @ y 
    log_prob = -0.5 * (n * np.log(np.pi) + p * np.log(1 + g) 
                       + (nu0 + n) * np.log(nu0 * sigma02 + SSRg) 
                       - nu0 * np.log(nu0 * sigma02)) + gammaln((nu0 + n) / 2) - gammaln(nu0 / 2)
    return log_prob

# Starting values and MCMC setup
S = 10000
Z = np.full((S, X.shape[1]), np.nan)
z = np.ones(X.shape[1])

# Initial log probability
lp_y_c = lp_y_X(y, X.iloc[:, z == 1])

# Gibbs sampler
for s in range(S):
    for j in np.random.permutation(X.shape[1]):
        zp = z.copy()
        zp[j] = 1 - zp[j]
        lp_y_p = lp_y_X(y, X.iloc[:, zp == 1])
        r = (lp_y_p - lp_y_c) * (-1) ** (zp[j] == 0)
        z[j] = np.random.binomial(1, 1 / (1 + np.exp(-r))) # whether to accept
        if z[j] == zp[j]:
            lp_y_c = lp_y_p
    Z[s, :] = z


### Metropolis-Hastings

In [7]:
X1 = df[['npreg', 'bp', 'bmi', 'ped', 'age']]

# centering and scaling
means = X1.mean()
stds = X1.std()
X_scaled = (X1 - means) / stds
X_scaled['intercept'] = 1
X_scaled = X_scaled[['intercept', 'npreg', 'bp', 'bmi', 'ped', 'age']].values


y1 = df['diabetes'].replace({'Yes': 1, 'No': 0})

/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_82243/3048451862.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y1 = df['diabetes'].replace({'Yes': 1, 'No': 0})


In [11]:
import numpy as np
from scipy.stats import multivariate_normal
from numpy.random import uniform

In [12]:
# Likelihood function
def likelihood(x):
    return np.exp(x) / (1 + np.exp(x))

In [13]:
# Initial values
n_iter = 10000
beta = np.ones(6)
gamma = np.ones(6)
standard = np.ones(len(y1))
p_beta = multivariate_normal.pdf(beta, np.zeros(6),
                                 np.diag([16, 4, 4, 4, 4, 4]))

In [ ]:
# Store samples
beta_samples = np.zeros((n_iter, 6))
gamma_samples = np.zeros((n_iter, 6))

# proporsal variance can be approximated by \sigma^2(X^{T} X)^{-1}, which is close to the posterior variance of \beta
var_prop = np.var(np.log(y1+1/2)) * np.linalg.inv(X_scaled.T @ X_scaled)

In [ ]:
# Metropolis-Hastings sampling
for i in range(n_iter):
    # Sample beta
    beta_prop = np.random.multivariate_normal(beta, var_prop)
    J_prop_beta = multivariate_normal.pdf(beta_prop, beta, var_prop)
    p_prop_beta = multivariate_normal.pdf(beta_prop, np.zeros(6),
                                          np.diag([16, 4, 4, 4, 4, 4]))
    J_beta = multivariate_normal.pdf(beta, beta_prop, var_prop)

    L_prop_beta = sum(np.log(likelihood(np.sum(beta_prop * gamma * X_scaled, axis=1))**y1
                             * (standard - likelihood(np.sum(beta_prop * gamma * X_scaled, axis=1)))**(1-y1)))
    
    L_beta = sum(np.log(likelihood(np.sum(beta * gamma * X_scaled, axis=1))**y1
                        * (standard - likelihood(np.sum(beta * gamma * X_scaled, axis=1)))**(1-y1)))

    r_log_beta = L_prop_beta + np.log(p_prop_beta) - L_beta - np.log(p_beta) + np.log(J_beta) - np.log(J_prop_beta)

    if np.log(uniform()) < min(0, r_log_beta):
        beta = beta_prop
        p_beta = p_prop_beta

    beta_samples[i] = beta

    # Sample gamma
    for r in np.random.permutation(range(1,6)):
        gamma_prop = gamma.copy()
        gamma_prop[r] = 1 - gamma_prop[r]

        L_prop_gamma = sum(np.log(likelihood(np.sum(beta * gamma_prop * X_scaled, axis=1))**y1
                                  * (standard - likelihood(np.sum(beta * gamma_prop * X_scaled, axis=1)))**(1-y1)))
        
        L_gamma = sum(np.log(likelihood(np.sum(beta * gamma * X_scaled, axis=1))**y1
                        * (standard - likelihood(np.sum(beta * gamma * X_scaled, axis=1)))**(1-y1)))

        r_log_gamma = L_prop_gamma - L_gamma

        if np.log(uniform()) < min(0, r_log_gamma):
            gamma = gamma_prop
    
    gamma_samples[i] = gamma

### Gibbs Sampling by Pólya-Gamma

In [14]:
import numpy as np
from polyagamma import random_polyagamma

# Initial values
n_iter = 10000
beta = np.ones(6)

# Store samples
beta_samples = np.zeros((n_iter, 6))
omega_samples = np.zeros((n_iter, len(y1)))
gamma_samples = np.zeros((n_iter, 6))


In [15]:
# Gibbs sampling by Pólya-Gamma
for i in range(n_iter):
    # Sample omega
    omega = np.zeros(len(y1))
    for r in range(len(y1)):
        x = X_scaled[r]
        omega[r] = random_polyagamma(1, np.dot(x, beta), size=1)
        
    omega_samples[i] = omega

    # Sample beta
    B = np.diag([16, 4, 4, 4, 4, 4])
    b = np.zeros(6)
    V_omega = np.linalg.inv(X_scaled.T @ np.diag(omega) @ X_scaled + np.linalg.inv(B)) # use of X_scaled
    k = y1 - np.ones(len(y1))/2
    m_omega = V_omega @ (X_scaled.T @ k + np.linalg.inv(B) @ beta)
    beta = np.random.multivariate_normal(m_omega, V_omega)
    beta_samples[i] = beta


    # Sample gamma
    for r in np.random.permutation(range(1,6)):
        gamma_prop = gamma.copy()
        gamma_prop[r] = 1 - gamma_prop[r]

        L_prop_gamma = sum(np.log(likelihood(np.sum(beta * gamma_prop * X_scaled, axis=1))**y1
                                  * (standard - likelihood(np.sum(beta * gamma_prop * X_scaled, axis=1)))**(1-y1)))
        
        L_gamma = sum(np.log(likelihood(np.sum(beta * gamma * X_scaled, axis=1))**y1
                        * (standard - likelihood(np.sum(beta * gamma * X_scaled, axis=1)))**(1-y1)))

        r_log_gamma = L_prop_gamma - L_gamma

        if np.log(uniform()) < min(0, r_log_gamma):
            gamma = gamma_prop
    
    gamma_samples[i] = gamma

/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_82243/3029163751.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  omega[r] = random_polyagamma(1, np.dot(x, beta), size=1)
/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_82243/3029163751.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  omega[r] = random_polyagamma(1, np.dot(x, beta), size=1)
/var/folders/dz/lgsl3l_d7fv4z40yrx07rfz80000gn/T/ipykernel_82243/3029163751.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation

In [26]:
np.diag(X_scaled[0])

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.44778583,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        , -0.28477392,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        , -0.39095814,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.40333095,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.70757816]])

In [34]:
X_scaled

array([[ 1.00000000e+00,  4.47785826e-01, -2.84773916e-01,
        -3.90958145e-01, -4.03330946e-01, -7.07578157e-01],
       [ 1.00000000e+00,  1.05164402e+00, -1.22307725e-01,
        -1.13211776e+00, -9.86706905e-01,  2.17303872e+00],
       [ 1.00000000e+00,  4.47785826e-01,  8.52489424e-01,
         4.22864176e-01, -1.00702348e+00,  3.14576218e-01],
       ...,
       [ 1.00000000e+00,  1.95743132e+00,  3.65090849e-01,
         1.42047398e-03, -9.63487961e-01,  2.91642372e+00],
       [ 1.00000000e+00,  4.47785826e-01,  4.01584665e-02,
        -9.72259803e-01, -7.48712733e-01, -1.50039407e-01],
       [ 1.00000000e+00, -7.59930572e-01, -1.22307725e-01,
        -3.61893062e-01, -5.45546976e-01, -8.00501282e-01]])

In [45]:
X_scaled.shape

(532, 6)

In [16]:
gamma_samples

array([[1., 0., 0., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1., 1.],
       ...,
       [1., 0., 0., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1., 1.]])

In [46]:
len(y1)

532

In [49]:
beta_samples

array([[-0.60959172,  0.72723408,  0.06789863,  0.99090325,  0.72825321,
         0.54073238],
       [-0.87285621,  0.4605069 ,  0.00292016,  0.89628469,  0.58844485,
         0.35105635],
       [-0.91790974,  0.35196829,  0.03729922,  0.66784841,  0.37003308,
         0.33285256],
       ...,
       [-0.89813027,  0.12589661,  0.04009773,  0.6134758 ,  0.37915628,
         0.62997468],
       [-0.95406702,  0.24004095,  0.17001577,  0.64669612,  0.42163814,
         0.56738406],
       [-0.90873408,  0.34694245,  0.16157717,  0.54719207,  0.38795935,
         0.26963518]])

In [17]:
sum(np.array([[1,2,3],[4,5,6]]))

array([5, 7, 9])

In [5]:
np.random.permutation(range(6))

array([3, 0, 5, 1, 2, 4])